In [1]:

import pandas as pd


In [2]:


from pymongo import MongoClient

url_servidor = 'mongodb://127.0.0.1:27017/'


client = MongoClient(url_servidor)

# código para ver si se ha conectado bien
try:
    s = client.server_info() # si hay error tendremos una excepción
    print("Conectado a MongoDB, versión",s["version"])
    db = client["PD1"]
except:
    print ("Error de conexión ¿está arrancado el servidor?")

Conectado a MongoDB, versión 8.2.3


In [3]:
df = pd.read_csv("MTA_Subway_Stations.csv")
df


,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,127,317,611,IRT,Broadway - 7Av,Times Sq-42 St,M,True,1 2 3,Subway,40.755290,-73.987495,Uptown,Downtown,1,1,1,NaN,POINT (-73.987495 40.75529)
1,S17,515,515,SIR,Staten Island,Annadale,SI,False,SIR,Open Cut,40.540460,-74.178217,Ferry,South Shore,0,0,0,NaN,POINT (-74.178217 40.54046)
2,S01,139,627,BMT,Franklin Shuttle,Franklin Av,Bk,False,S,Elevated,40.680596,-73.955827,Last Stop,Prospect Park,1,1,1,NaN,POINT (-73.955827 40.680596)
3,254,349,349,IRT,Eastern Pky,Junius St,Bk,False,3,Elevated,40.663515,-73.902447,Manhattan,New Lots,0,0,0,NaN,POINT (-73.902447 40.663515)
4,M01,108,108,BMT,Myrtle Av,Middle Village-Metropolitan Av,Q,False,M,Elevated,40.711396,-73.889601,Inbound,Last Stop,1,1,1,NaN,POINT (-73.889601 40.711396)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,B06,222,222,IND,63rd St,Roosevelt Island,M,False,M,Subway,40.759145,-73.953260,Queens,Downtown,1,1,1,NaN,POINT (-73.95326 40.759145)
492,J15,83,83,BMT,Jamaica,Woodhaven Blvd,Q,False,J Z,Elevated,40.693879,-73.851576,Jamaica,Manhattan,1,1,1,NaN,POINT (-73.851576 40.693879)
493,S18,514,514,SIR,Staten Island,Eltingville,SI,False,SIR,Embankment,40.544601,-74.164570,Ferry,South Shore,0,0,0,NaN,POINT (-74.16457 40.544601)
494,F14,232,232,IND,6th Av - Culver,2 Av,M,True,F,Subway,40.723402,-73.989938,Uptown,Downtown,0,0,0,NaN,POINT (-73.989938 40.723402)


In [4]:

columnas_utiles = ['GTFS Stop ID', 'Stop Name', 'Daytime Routes', 'GTFS Longitude', 'GTFS Latitude']
df_limpio = df[columnas_utiles].copy()


df_limpio = df_limpio.rename(columns={
    'GTFS Stop ID': 'GTFS_id_estacion',
    'Stop Name': 'nombre',
    'Daytime Routes': 'lineas'
})


df_limpio['ubicacion'] = df_limpio.apply(
    lambda fila: {
        "type": "Point",
        "coordinates": [fila['GTFS Longitude'], fila['GTFS Latitude']]
    }, axis=1
)


df_limpio = df_limpio.drop(columns=['GTFS Longitude', 'GTFS Latitude'])


documentos_para_mongo = df_limpio.to_dict(orient='records')

In [5]:
df_limpio

,GTFS_id_estacion,nombre,lineas,ubicacion
0,127,Times Sq-42 St,1 2 3,"{'type': 'Point', 'coordinates': [-73.987495, ..."
1,S17,Annadale,SIR,"{'type': 'Point', 'coordinates': [-74.178217, ..."
2,S01,Franklin Av,S,"{'type': 'Point', 'coordinates': [-73.955827, ..."
3,254,Junius St,3,"{'type': 'Point', 'coordinates': [-73.902447, ..."
4,M01,Middle Village-Metropolitan Av,M,"{'type': 'Point', 'coordinates': [-73.889601, ..."
...,...,...,...,...
491,B06,Roosevelt Island,M,"{'type': 'Point', 'coordinates': [-73.95326, 4..."
492,J15,Woodhaven Blvd,J Z,"{'type': 'Point', 'coordinates': [-73.851576, ..."
493,S18,Eltingville,SIR,"{'type': 'Point', 'coordinates': [-74.16457, 4..."
494,F14,2 Av,F,"{'type': 'Point', 'coordinates': [-73.989938, ..."


In [6]:
db.subway.drop()

In [7]:
db.subway.insert_many(documentos_para_mongo)


InsertManyResult([ObjectId('6995d06fa4ed9b5cfd573d05'), ObjectId('6995d06fa4ed9b5cfd573d06'), ObjectId('6995d06fa4ed9b5cfd573d07'), ObjectId('6995d06fa4ed9b5cfd573d08'), ObjectId('6995d06fa4ed9b5cfd573d09'), ObjectId('6995d06fa4ed9b5cfd573d0a'), ObjectId('6995d06fa4ed9b5cfd573d0b'), ObjectId('6995d06fa4ed9b5cfd573d0c'), ObjectId('6995d06fa4ed9b5cfd573d0d'), ObjectId('6995d06fa4ed9b5cfd573d0e'), ObjectId('6995d06fa4ed9b5cfd573d0f'), ObjectId('6995d06fa4ed9b5cfd573d10'), ObjectId('6995d06fa4ed9b5cfd573d11'), ObjectId('6995d06fa4ed9b5cfd573d12'), ObjectId('6995d06fa4ed9b5cfd573d13'), ObjectId('6995d06fa4ed9b5cfd573d14'), ObjectId('6995d06fa4ed9b5cfd573d15'), ObjectId('6995d06fa4ed9b5cfd573d16'), ObjectId('6995d06fa4ed9b5cfd573d17'), ObjectId('6995d06fa4ed9b5cfd573d18'), ObjectId('6995d06fa4ed9b5cfd573d19'), ObjectId('6995d06fa4ed9b5cfd573d1a'), ObjectId('6995d06fa4ed9b5cfd573d1b'), ObjectId('6995d06fa4ed9b5cfd573d1c'), ObjectId('6995d06fa4ed9b5cfd573d1d'), ObjectId('6995d06fa4ed9b5cfd573d

In [8]:
from rich import print_json
from json import dumps

# muestra el contenido de un cursor
def muestra(cursor):
    # Convertir a JSON y mostrar con rich
    i = 0
    for doc in cursor:
        i+=1
        print_json(dumps(doc, indent=4, default=str))
    return i

In [9]:
muestra(db.subway.find({}).limit(1))

{
  "_id": "6995d06fa4ed9b5cfd573d05",
  "GTFS_id_estacion": "127",
  "nombre": "Times Sq-42 St",
  "lineas": "1 2 3",
  "ubicacion": {
    "type": "Point",
    "coordinates": [
      -73.987495,
      40.75529
    ]
  }
}

1

In [10]:
db.subway.drop_indexes()

In [11]:
db.subway.create_index({"ubicacion":"2dsphere"})

'ubicacion_2dsphere'

In [12]:

def cursor_paradas_afectedas(coordinates): #coordinates de esta forma [longitud, latitud]
    cursor = db.subway.find(
   {
     "ubicacion":
       { "$near" :
          {
            "$geometry": { "type": "Point",  "coordinates": coordinates },
            "$maxDistance": 500
          }
       }
   }
)
    return cursor


#muestra(cursor)

In [14]:
def extraccion_paradas(cursor):
    afectadas = []
    for doc in cursor:
        afectadas.append((doc["nombre"], doc["lineas"]))

    return afectadas


In [15]:
#ejemplo
afectadas = extraccion_paradas(cursor_paradas_afectedas([-73.9866, 40.7592]))
afectadas

[('49 St', 'N R W'),
 ('42 St-Port Authority Bus Terminal', 'A C E'),
 ('Times Sq-42 St', 'S'),
 ('50 St', '1'),
 ('50 St', 'C E'),
 ('Times Sq-42 St', '7'),
 ('Times Sq-42 St', '1 2 3'),
 ('47-50 Sts-Rockefeller Ctr', 'B D F M')]